# Proyecto Final: Asistente IA para Seguros del Hogar

**Autor:** *Joaquín Hudson*

**Curso:** *Inteligencia artificial: Generación de Prompts*

**Comisión:** *67115*


## Introducción
En este proyecto se desarrolla un asistente de inteligencia artificial que actúa como asesor de seguros del hogar. El asistente permite a los usuarios cargar su póliza de seguro y describir un evento. Con esta información, analiza la cobertura del seguro y proporciona una respuesta clara sobre los montos cubiertos.

**Propósito:** Simplificar la interpretación de pólizas de seguro, mejorando la experiencia del usuario y reduciendo la incertidumbre en situaciones de siniestro.


## Metodología
El proyecto sigue los siguientes pasos:
1. Carga de la póliza de seguro en formato PDF o texto plano (.txt).
2. Preprocesamiento del texto para segmentar secciones relevantes.
3. Análisis de consultas del usuario mediante técnicas de Fast Prompting con GPT-4.
4. Respuesta clara y detallada sobre la cobertura.

**Herramientas:**

- Modelo GPT-4 para análisis de texto y generación de respuestas.
- Modelo Dall-e para generacion de imagen de asistentes.
- Python para preprocesamiento de datos.
- Jupyter Notebook como entorno de desarrollo.

**Tecnicas de fast rpompting:**

- In-Context Lerning y asignacion de roles:Se da un contexto claro dando instrucciones explicitas, por un lado se entrega la poliza en la que debe bazar su respuesta y por el otro se le da una instruccion general en la que es un asistente experto en seguros del hogar. De esta manera se evitan respuestas no relacionadas con el tema y se da especificidad.


- Prompting dirigido: Utiliza una estructura predefinida para la respuesta que guia resultados consistentespara garantizar que las respuestas sean claras y relevantes. Facilita la comprensión del usuario al organizar la información de manera lógica y estructurada. Además, garantiza consistencia en las respuestas, mejorando la experiencia de usuario.

## Implementación

In [1]:
import openai
import os
from PyPDF2 import PdfReader
import requests
from IPython.display import Image, display
# Configuración de la API de OpenAI
openai.api_key = "Ingrear Apikey"


### Función para cargar y preprocesar la póliza de seguro desde PDF o texto plano


In [2]:

def cargar_poliza(ruta_archivo):
    """
    Carga el texto de una póliza de seguro desde un archivo local o una URL.
    """
    if ruta_archivo.startswith("http"):  # Detecta si es una URL
        response = requests.get(ruta_archivo)
        if response.status_code == 200:
            ruta_local = "temp_poliza.pdf"
            with open(ruta_local, "wb") as f:
                f.write(response.content)
            ruta_archivo = ruta_local
        else:
            raise ValueError("No se pudo descargar el archivo. Verifica la URL.")

    if ruta_archivo.endswith('.pdf'):
        lector = PdfReader(ruta_archivo)
        texto = ''
        for pagina in lector.pages:
            texto += pagina.extract_text()
    elif ruta_archivo.endswith('.txt'):
        with open(ruta_archivo, 'r', encoding='utf-8') as file:
            texto = file.read()
    else:
        raise ValueError("Formato de archivo no soportado. Usa PDF o TXT.")
    
    return texto

### Función para validar el texto de la póliza


In [3]:
def validar_poliza(poliza_texto):
    """
    Valida si el texto cargado como póliza contiene contenido suficiente para analizar.
    """
    if len(poliza_texto) < 500:
        raise ValueError("La póliza cargada parece ser demasiado corta o estar incompleta.")
    if not any(keyword in poliza_texto.lower() for keyword in ["cobertura", "exclusión", "siniestro"]):
        raise ValueError("El texto de la póliza no contiene palabras clave esperadas.")
    return True

### Función para analizar consultas del usuario


In [4]:
def analizar_evento(poliza_texto):
    """
    Interactúa con el usuario para capturar el evento, lo analiza y devuelve si está cubierto.
    """
    imagen_url = generar_imagen_asistente()
    # Validar el texto de la póliza
    validar_poliza(poliza_texto)

    # Limitar el texto relevante de la póliza
    if len(poliza_texto) > 2000:
        poliza_texto = poliza_texto[:2000] + "..."  # Limitar a los primeros 2000 caracteres

    # Capturar el evento descrito por el usuario
    evento = input("Describe qué te pasó: ")

    # Mensajes para el modelo de chat
    mensajes = [
        {
            "role": "system",
            "content": (
                "Eres un asistente experto en seguros del hogar y con conocimientos de derecho argentino. Analiza eventos en función de una póliza proporcionada, "
                "Es importante tener en cuenta en la respuesta las cláusulas especiales, exclusiones y pre-requisitos necesarios para cobertura. "
                "Responde principalmente basándote en la póliza."
            )
        },
        {
            "role": "user",
            "content": (
                f"Poliza del Seguro:\n{poliza_texto}\n\n"
                f"Evento Descrito:\n{evento}\n\n"
                "Responde de manera concisa:\n"
                "1. ¿El evento está cubierto según la póliza?\n"
                "2. Si está cubierto, lista los pre-requisitos necesarios para hacer efectiva la cobertura.\n"
                "3. Si no está cubierto, indica brevemente por qué no y señala las exclusiones relevantes.\n"
            )
        }
    ]

    # Llamada al modelo GPT en formato de chat
    respuesta = openai.ChatCompletion.create(
        model="gpt-4",
        messages=mensajes,
        max_tokens=300  # Limitar la longitud de la respuesta
    )

    return respuesta.choices[0].message['content'].strip()

# Generacion de Imagen

In [5]:
def generar_imagen_asistente():
    """
    Genera una imagen de un asistente de seguros utilizando la API de DALL-E.
    """
    prompt = "A friendly, professional insurance assistant with a confident smile, wearing formal business attire. The assistant is standing in a modern office with a laptop in front of them, ready to help a client with their home insurance needs."
    
    # Generación de imagen con DALL-E
    respuesta = openai.Image.create(
        prompt=prompt,
        n=1,
        size="1024x1024"
    )
    
    # URL de la imagen generada
    url_imagen = respuesta['data'][0]['url']
    print(f"Imagen generada: {url_imagen}")
    return url_imagen
 # Mostramos la Imagen
def mostrar_imagen(imagen_url):
    """
    Muestra la imagen generada del asistente de seguros en el Jupyter Notebook.
    """
    try:
        display(Image(url=imagen_url, width=400, height=400))
    except Exception as e:
        print(f"Error al mostrar la imagen: {e}")
        print("Puedes abrir la imagen directamente desde esta URL:", imagen_url)


# Ejemplo de uso

In [6]:
ruta_poliza = "https://drive.google.com/uc?id=1uwNaJ2M2UM7Xua7_qHJal2BEW_eivQmz&export=download"  
poliza_texto = cargar_poliza(ruta_poliza)

respuesta = analizar_evento(poliza_texto)

print("Respuesta del asistente:")
print(respuesta)

Imagen generada: https://oaidalleapiprodscus.blob.core.windows.net/private/org-tHP0VY7N0RnV2heDPlMaRuD1/user-ZyJkjnjk7gUcGUtXndRUTj32/img-BGlYvKx8jfp4bgWRb95sezIY.png?st=2024-12-20T22%3A04%3A56Z&se=2024-12-21T00%3A04%3A56Z&sp=r&sv=2024-08-04&sr=b&rscd=inline&rsct=image/png&skoid=d505667d-d6c1-4a0a-bac7-5c84a87759f8&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2024-12-20T19%3A40%3A31Z&ske=2024-12-21T19%3A40%3A31Z&sks=b&skv=2024-08-04&sig=e9Rv84bVjbguP2/CkWNwkEE7DA/l7HxnmxgwSduxCkI%3D
Describe qué te pasó: incendio de hogar
Respuesta del asistente:
1. Sí, el evento de incendio de hogar está cubierto según la póliza, siempre que no esté dentro de las exclusiones mencionadas.

2. Para hacer efectiva la cobertura se necesitaría:

   - Prueba de que el incendio ocurrió, sin haber sido causado por un vicio propio de la cosa (según la exclusión A)
   - Evidencia de que el incendio no fue causado por terremoto, meteorito, maremoto o erupción volcánica, tornado, huracán o ciclón, inundación, a

## Resultados y Conclusiones
**Resultados:**
1. El asistente fue capaz de identificar exclusiones importantes en eventos específicos, según el texto de la póliza.
2. En casos de cobertura, proporcionó los pre-requisitos necesarios, mejorando la claridad para el usuario.
3. Facilita el trabajo del productor de seguros al interpretar la poliza de manera agil respondiendo situaciones simples

**Conclusiones:**
- Este proyecto demuestra la viabilidad de usar IA para interpretar pólizas complejas, reduciendo la ambigüedad y ayudando a los usuarios a tomar decisiones informadas.
- En el futuro, se podría incluir una integración más profunda con bases de datos de aseguradoras para automatizar el análisis de siniestros.
- Tambien podria incorporar Prompting Iterativo para refinar el prompt en relacion a su uso.
